In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the folder path containing your folders of images
folder_path = "R:\SIEMENS_Intern\chlorine"

# List all folders in the specified directory
folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

# Define the labels
labels = [0.1, 0.2, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0]

# Create a dictionary to store the mapping of folder names to labels
label_mapping = {folder: label for folder, label in zip(folders, labels)}

# Print the mapping to verify
print("Folder Label Mapping:")
print(label_mapping)



Folder Label Mapping:
{'Sample_0.1': 0.1, 'Sample_0.2': 0.2, 'Sample_0.5': 0.5, 'Sample_1.0': 1.0, 'Sample_1.5': 1.5, 'Sample_2.0': 2.0, 'Sample_2.5': 2.5, 'Sample_3.0': 3.0, 'Sample_4.0': 4.0, 'Sample_5.0': 5.0}


In [2]:
# Define image dimensions and batch size
img_height, img_width = 224, 224
batch_size = 32

# Define train and validation data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Splitting data into train and validation
train_generator = train_datagen.flow_from_directory(
    folder_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')  # Set as training data

validation_generator = train_datagen.flow_from_directory(
    folder_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')  # Set as validation data

Found 1591 images belonging to 10 classes.
Found 392 images belonging to 10 classes.


In [3]:

from tensorflow.keras.applications import ResNet50

# Load pre-trained ResNet50 model without top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of ResNet50
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
predictions = layers.Dense(len(labels), activation='softmax')(x)

# Create the model
model = models.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Save the trained model
model.save("resnet50_chlorine_concentration_model.h5")

Epoch 1/10
49/49 [==============================] - 242s 5s/step - loss: 2.3639 - accuracy: 0.0885 - val_loss: 2.3590 - val_accuracy: 0.1016
Epoch 2/10
49/49 [==============================] - 219s 4s/step - loss: 2.3284 - accuracy: 0.1026 - val_loss: 2.3346 - val_accuracy: 0.1276
Epoch 3/10
49/49 [==============================] - 218s 4s/step - loss: 2.3335 - accuracy: 0.1090 - val_loss: 2.3308 - val_accuracy: 0.0990
Epoch 4/10
49/49 [==============================] - 213s 4s/step - loss: 2.3143 - accuracy: 0.1193 - val_loss: 2.3044 - val_accuracy: 0.1094
Epoch 5/10
49/49 [==============================] - 202s 4s/step - loss: 2.3083 - accuracy: 0.1065 - val_loss: 2.3563 - val_accuracy: 0.0964
Epoch 6/10
49/49 [==============================] - 200s 4s/step - loss: 2.3080 - accuracy: 0.1232 - val_loss: 2.2986 - val_accuracy: 0.1146
Epoch 7/10
49/49 [==============================] - 201s 4s/step - loss: 2.3097 - accuracy: 0.1199 - val_loss: 2.2962 - val_accuracy: 0.1458
Epoch 8/10
49

In [10]:
# Classify new images
new_image_path = r'R:\SIEMENS_Intern\chlorine\Sample_2.5\aug_0_200.jpeg'
img = tf.keras.preprocessing.image.load_img(new_image_path, target_size=(img_height, img_width))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create a batch

predictions = model.predict(img_array)
predicted_class_index = tf.argmax(predictions, axis=1)
predicted_class = labels[predicted_class_index[0]]
print("Predicted concentration:", predicted_class)

1/1 [==============================] - 0s 276ms/step
Predicted concentration: 0.1
